<a href="https://colab.research.google.com/github/INF800/nlp_projects/blob/main/qa_chatbot_information_retrieval/qa_chatbot_information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Mon Aug 16 20:28:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git -qqq

     |████████████████████████████████| 2.5 MB 7.0 MB/s 
     |████████████████████████████████| 204 kB 7.0 MB/s 
     |████████████████████████████████| 52 kB 949 kB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 321 kB 69.5 MB/s 
     |████████████████████████████████| 330 kB 73.1 MB/s 
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 981 kB 65.6 MB/s 
     |████████████████████████████████| 5.6 MB 49.3 MB/s 
     |████████████████████████████████| 100 kB 12.1 MB/s 
     |████████████████████████████████| 8.4 MB 62.1 MB/s 
     |████████████████████████████████| 402 kB 66.6 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 50 kB 8.8 MB/s 
     |████████████████████████████████| 56 kB 6.2 MB/s 
     |████████████████████████████████| 51.5 MB 20 kB/s 
     |████████████████████████

In [2]:
!pip install wikipedia -qqq

In [3]:
# BUILDING KNOWLEDGE BASE (this is not training data)

In [18]:
from pathlib import Path
import wikipedia as wik

TOPIC = 'Narendra Modi' # give the topic you want to build chatbot for
N_DOCS = 30 # minimum 600

write_dir = Path('data') / f'wiki_txt_{TOPIC}'
write_dir.mkdir(exist_ok=True, parents=True)

wik.set_lang("en")
page_names = wik.search(TOPIC, results=N_DOCS)
cntr = 0
for name in page_names:
  try:
    pg = wik.page(name)
    with open(str(write_dir/name)+'.txt', 'w') as f:
      f.write(pg.content)

    cntr+=1
    print('✔ added to knowledge base:', name)
  except:
    print('❌could not process:', name)

print('added total:', cntr, 'wiki pages')

✔ added to knowledge base: Narendra Modi
✔ added to knowledge base: Narendra Modi Stadium
✔ added to knowledge base: PM Narendra Modi
✔ added to knowledge base: Foreign policy of the Narendra Modi government
✔ added to knowledge base: Premiership of Narendra Modi
✔ added to knowledge base: List of international prime ministerial trips made by Narendra Modi
✔ added to knowledge base: Jashodaben Modi
✔ added to knowledge base: Opinion polling on the Narendra Modi premiership
✔ added to knowledge base: Public image of Narendra Modi
✔ added to knowledge base: Bibliography of Narendra Modi
✔ added to knowledge base: First Modi ministry
✔ added to knowledge base: Modi ministry
✔ added to knowledge base: Sanjay Joshi
✔ added to knowledge base: Second Modi ministry
✔ added to knowledge base: Electoral history of Narendra Modi
✔ added to knowledge base: List of nicknames of the prime ministers of India
✔ added to knowledge base: List of things named after the prime ministers of India
✔ added to

In [12]:
# SETUP HAYSTACK

In [13]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [14]:
# Note: Does not work in colab
# # Recommended: Start Elasticsearch using Docker via the Haystack utility function
# from haystack.utils import launch_es

# launch_es()

In [15]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [16]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

08/16/2021 21:20:58 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.011s]
08/16/2021 21:20:58 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.008s]
08/16/2021 21:20:58 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.006s]
08/16/2021 21:20:58 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.018s]
08/16/2021 21:20:58 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store

In this tutorial, we download Wikipedia articles about Game of Thrones, apply a basic cleaning function, and index them in Elasticsearch.

In [19]:
# Path to knowledge base
doc_dir = str(write_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'text': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
#}
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(dicts[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Afghanistan–India relations.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Narendra Modi Stadium.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Sanjiv Bhatt.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/List of nicknames of the prime ministers of India.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Union Council of Ministers.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Opinion polling on the Narendra Modi premiership.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Converting data/wiki_txt_Narendra Modi/Sanjay Joshi.txt
08/16/2021 21:22:11 - INFO - haystack.preprocessor.utils -   Convertin

[{'text': 'Afghanistan and India have remained strong and friendly over the decades. They had been historical neighbours, and share cultural ties through Bollywood and cricket.The Republic of India was the only South Asian country to recognize the Soviet-backed Democratic Republic of Afghanistan in the 1980s, though relations were diminished during the 1990s Afghan civil war and the Taliban government. India aided the overthrow of the Taliban and became the largest regional provider of humanitarian and reconstruction aid to the present day Islamic Republic of Afghanistan. Indians are working in various construction projects, as part of India\'s rebuilding efforts in Afghanistan. Pakistan alleges the India\'s foreign intelligence agency R&AW is working in cover to malign Pakistan and train and support insurgents, a claim rejected strongly by India.Shaida Mohammad Abdali, Afghanistan\'s ambassador to India, in April 2017 pointed out that India "is the biggest regional donor to Afghanista

08/16/2021 21:22:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.228s]


## Initalize Retriever, Reader,  & Pipeline

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

In [20]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [21]:
# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.retriever.sparse import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)

### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

#### FARMReader

In [22]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

08/16/2021 21:22:36 - INFO - farm.utils -   Using device: CUDA 
08/16/2021 21:22:36 - INFO - farm.utils -   Number of GPUs: 1
08/16/2021 21:22:36 - INFO - farm.utils -   Distributed Training: False
08/16/2021 21:22:36 - INFO - farm.utils -   Automatic Mixed Precision: None
08/16/2021 21:22:36 - INFO - filelock -   Lock 140655588346768 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

08/16/2021 21:22:36 - INFO - filelock -   Lock 140655588346768 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock
08/16/2021 21:22:37 - INFO - filelock -   Lock 140658662486928 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

08/16/2021 21:22:48 - INFO - filelock -   Lock 140658662486928 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock
Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['ro

...

08/16/2021 21:23:01 - INFO - filelock -   Lock 140655444634704 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
08/16/2021 21:23:01 - INFO - filelock -   Lock 140655586858512 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

08/16/2021 21:23:02 - INFO - filelock -   Lock 140655586858512 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
08/16/2021 21:23:02 - INFO - filelock -   Lock 140655432951632 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

08/16/2021 21:23:02 - INFO - filelock -   Lock 140655432951632 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
08/16/2021 21:23:03 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
08/16/2021 21:23:03 - INFO - farm.utils -   Using device: CUDA 
08/16/2021 21:23:03 - INFO - farm.utils -   Number of GPUs: 1
08/16/2021 21:23:03 - INFO - farm.utils -   Distributed Training: False
08/16/2021 21:23:03 - INFO - farm.utils -   Automatic Mixed Precision: None
08/16/2021 21:23:03 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
08/16/2021 21:23:03 - INFO - farm.infer -    0    0 
08/16/2021 21:23:03 - INFO - farm.infer -   /w\  /w\
08/16/2021 21:23:03 - INFO - farm.infer -   /'\  / \
08/16/2021 21:23:03 - INFO - farm.infer -     
08/16/2021 21:23:03 - INFO - farm.utils -   Usi

#### TransformersReader

In [ ]:
# Alternative:
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `ExtractiveQAPipeline` that combines a retriever and a reader to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).

In [23]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

## Voilà! Ask a question!

In [29]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = pipe.run(query="When was PM modi born", top_k_retriever=10, top_k_reader=5)

print("\n")
print("\n")
print_answers(prediction, details="minimal")

08/16/2021 21:25:50 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 46.76 Batches/s]





[   {   'answer': '17 September 1950',
        'context': '\n'
                   '== Early life and education ==\n'
                   'Narendra Modi was born on 17 September 1950 to a Gujarati '
                   'Hindu family of grocers in Vadnagar, Mehsana district, '
                   'Bomba'},
    {   'answer': 'January 1952',
        'context': 'Jashodaben Narendra Modi (née Jashodaben Chimanlal Modi; '
                   'born January 1952) is a retired Indian school teacher and '
                   'estranged wife of Narendra Modi. Th'},
    {   'answer': '1947',
        'context': "rime Minister born after India's independence from the "
                   'British Empire in 1947. Modi started his second term after '
                   'the Bharatiya Janata Party led Natio'},
    {   'answer': 'after independence',
        'context': 't he was going to be the first Prime Minister of India to '
                   'be born after independence and that it would be natural t

## Credits:

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Slack](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://apply.workable.com/deepset/) 
